### Testing notebook for the end-to-end simulator (old)

In [ ]:
# Regular imports

import numpy as np
import scipy.special as sps
import scipy.constants as spc
import scipy.interpolate as spinter
import scipy.optimize as spopt
import scipy.fft as ft
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
from PIL import Image


In [ ]:
# Importing the Modules, REDO when making any changes to them or coppy here

import images
import instrument
import process
import analysis

In [ ]:

def Correct_diff_fringes():

    # energy in keV: 1.2398425keV or 10A from Willingale (2004),
    # 3.6 keV found as optimum by Emily in her Master thesis,
    # and 6.4 keV the iron K-alpha line
    energy = 1.2398425 #3.6 #6.4

    # seemingly arbetrary time step?
    time_step = 1

    # the chosen number of source counts
    photons_counts = int(1e5)

    # making the interferometer itself
    test_I = instrument.interferometer(1, roller = instrument.interferometer.smooth_roller, roll_speed = 0)
    
    # adding a baseline, values based on discussions
    test_I.add_baseline(D = 1, L = 10, W = 400, num_pairs = 5)

    # taking the standard detector, it can be changed
    detector = test_I.baselines[0].camera

    # binning the entire detector, based on the detector charateristics
    bins = int(np.ceil(abs(detector.pos_range[0] - detector.pos_range[1]) / detector.res_pos))
    edges = np.array([detector.pos_range[0] + i * detector.res_pos for i in range(bins + 1)]) * 1e6

    # creating the source
    image = images.point_source(photons_counts, 0.000, 0.0009, energy)

    # observing the source using the optical bench
    test_data = process.interferometer_data(test_I, image, photons_counts)

    # plotting the binned photon positions
    plt.hist(test_data.pos * 1e6, 1000, label=f'{energy} keV')
    plt.xlabel('Photon impact positions ($\\mu$m)')
    plt.ylabel('Number of photons in bin')
    plt.ylim(0, 2200) #3500) #
    plt.show()


Correct_diff_fringes()


In [ ]:

def image_reconstruction():

    def fourier2D(matrix_in):
        """
        Calculate the 2D Fourier transform of a matrix

        Input:
            matrix_in: 2D numpy array
        
        Output:
            matrix_out: 2D Fourier transform of matrix_in
        """
    
        matrix_out = np.absolute(np.fft.fft2(matrix_in))
        matrix_out = np.roll(matrix_out, int(matrix_out.shape[0]/2), axis=0)
        matrix_out = np.roll(matrix_out, int(matrix_out.shape[1]/2), axis=1)
        return matrix_out


    """Point source"""
    # offset = 0.0009
    # image = images.point_source(int(1e7), 0.000, offset, 1.2398425) # image = images.m_point_sources(int(1e6), 4, [0.000, -0.000, -.0004, .00085], [0.000236, -0.00065, 0., 0.], [1.2, 1.2, 1.2, 1.2])
    # # img_scale is unclear to me what it's based on. I presume it to be based on pixel size compared to source size
    # img_scale = 1
    # pix_scale =  np.array([330, 550])

    """All models are those from the dropbox 'models'."""

    """Code for a plot of cyg X-1"""
    image_path = r"Models\hmxb.jpg"
    img_scale = .00055

    """Code for AU mic""" 
    # Image is big though, so expect a long wait
    # image_path = r"au_mic.png"
    # img_scale = 0.0013

    """Code for sgr A*"""
    # Remember to add // 5 to pix_scale to make sure there aren't too many useless pixels taken into account
    # image_path = r"bhdisk.png"
    # img_scale = 0.00037

    """Code for 'compact' model"""
    # image_path = r"compact.png"
    # img_scale = 0.000512 * 2

    """This samples the source, when an image is given"""
    image, pix_scale = images.generate_from_image(image_path, int(1e5), img_scale, 1.2398425)#6.4)#3.6)#, energy_spread = 0.1)#3.6)#6.4)#

    # plots the sampled points of origin of the source photons
    plt.plot(image.loc[:,1] * (3600*360 / (2 * np.pi)), -image.loc[:,0] * (3600*360 / (2 * np.pi)), '.', alpha=.2)
    histed_photons, _, __ = np.histogram2d(image.loc[:,0], image.loc[:,1], pix_scale)
    plt.imshow(histed_photons, cmap=cm.Greens)
    plt.show()

    # seemingly arbetrary time step?
    time_step = 1
    
    # making the interferometer itself
    test_I = instrument.interferometer(time_step, roller = instrument.interferometer.smooth_roller,
                                       roll_init = 0, roll_speed = np.pi / (np.max(image.toa) * time_step)) #.00001 * 2 * np.pi) #
    
    # adding the baselines, values based trying different things
    # baseline_Ds = [0.001, 0.01, 0.1, 0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    for D in np.linspace(0.05, 1, 10): #baseline_Ds: #np.logspace(0.05, 1, 10):
        test_I.add_baseline(num_pairs = 5, D = D, L = 10, W = 300)
        # test_I.add_baseline(num_pairs = 5, D = D, L = 10, W = 400) # most realistic atm based on discussions
        # test_I.add_willingale_baseline(D)

    # observing the source using the optical benches and timing it
    start = time.time()
    # potentially adding positional and energy noise
    test_data = process.interferometer_data(test_I, image)#, 2, .15 / (2 * np.sqrt(2*np.log(2))))
    print('Processing this image took ', time.time() - start, ' seconds')

    # reconstruction of the source and timing it
    start = time.time()
    re_im, uv = analysis.image_recon_smooth(test_data, test_I, .02 * 2 * np.pi, img_scale, samples = pix_scale) 
    print('Reconstructing this image took ', time.time() - start, ' seconds')

    # showing the sampled points in the uv-plane
    fig = plt.figure(figsize=(6,6))
    plt.plot(uv[:, 0], uv[:, 1], 'g.')
    plt.xlim(-np.max(uv) * 1.2, np.max(uv) * 1.2)
    plt.ylim(-np.max(uv) * 1.2, np.max(uv) * 1.2)
    plt.title("uv-plane sampling")
    plt.show()

    # showing the log of the Fourier transform of the reconstructed image
    fig = plt.figure(figsize=(6,6))
    plt.imshow(np.log10(fourier2D(re_im)), cmap=cm.cubehelix)
    plt.colorbar()
    plt.show()

    # showing the reconstructed image
    fig = plt.figure(figsize=(6,6))
    plt.imshow(re_im, cmap=cm.cubehelix)
    plt.colorbar()
    plt.xlabel('x ($\mu$as)')
    plt.ylabel('y ($\mu$as)')
    plt.show()

    # showing the log of the reconstructed image
    fig = plt.figure(figsize=(6,6))
    plt.imshow(np.log10(re_im), cmap=cm.cubehelix)
    plt.colorbar()
    plt.xlabel('x ($\mu$as)')
    plt.ylabel('y ($\mu$as)')
    plt.show()

image_reconstruction()


In [ ]:

def test_periodgram():

    # energy in keV: 1.2398425keV or 10A from Willingale (2004),
    # 3.6 keV found as optimum by Emily in her Master thesis,
    # and 6.4 keV the iron K-alpha line
    energy = 1.2398425 #3.6 #6.4

    # making the interferometer itself
    test_I = instrument.interferometer(1, roller = instrument.interferometer.smooth_roller, roll_speed = 0)
    
    # adding a baseline, values based on discussions
    test_I.add_baseline(D = 1, L = 10, W = 400, num_pairs = 5)

    # taking the standard detector, it can be changed
    detector = test_I.baselines[0].camera

    # creating the source
    image = images.point_source(int(1e5), 0.000, 0.0009, energy)
    
    # observing the source using the optical bench
    test_data = process.interferometer_data(test_I, image)
    
    # binning the entire detector, based on the detector charateristics
    bins = int(np.ceil(abs(detector.pos_range[0] - detector.pos_range[1]) / detector.res_pos))
    edges = np.array([detector.pos_range[0] + i * detector.res_pos for i in range(bins + 1)]) * 1e6

    # plotting the binned photon positions
    plt.hist(test_data.pos * 1e6, edges, label=f'{energy} keV')
    plt.xlabel('Photon impact positions ($\\mu$m)')
    plt.ylabel('Number of photons in bin')
    plt.ylim(0, 2200)
    plt.show()

    # making a periodogram of the above photon positions
    analysis.periodogram(test_data, detector, test_data.pos)

test_periodgram()
